In [0]:
#Google Colab needs
import os
import sys
def isCollab():
    return os.environ.get('COLAB_GPU', None) != None

if isCollab():
    #Mounting GDrive disc
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    path_to_storage = '/content/gdrive/My Drive/UCU-2019-final-project-storage'

    #Append path where custom modules stored. I put custom modules to GDrive disc
    path_to_modules = '/content/gdrive/My Drive/UCU-2019-final-project-storage'
    sys.path.append(path_to_modules)
else:
    sys.path.append('..')
    path_to_storage = None

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import keras
import pickle
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
import re
from keras import optimizers
from keras import activations
from keras import losses
from keras import metrics
from keras.utils import to_categorical
from keras.models import load_model
from keras import layers
from keras import activations
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Input
import gensim
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_multiple_whitespaces, remove_stopwords, stem_text
from keras.models import Model
import keras.backend as K
import nltk

Using TensorFlow backend.


In [0]:
nltk.download('stopwords')
from nltk.corpus import stopwords

stops = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
if not path_to_storage:
    path_to_storage = os.path.abspath(os.path.join(os.getcwd(), '../storage')) 

path_to_objects = path_to_storage + "/serialization_objects/"
data_folder = path_to_storage+'/data/'
X_train = pickle.load(open(path_to_objects + 'X_train.p', 'rb'))
y_train = pickle.load(open(path_to_objects + 'y_train.p', 'rb'))
X_test = pickle.load(open(path_to_objects + 'X_test.p', 'rb'))
y_test = pickle.load(open(path_to_objects + 'y_test.p', 'rb'))

In [0]:
q1_vecs_train = pickle.load(open(path_to_objects + 'X_train_q1_w2v_vect.p', 'rb'))
q2_vecs_train = pickle.load(open(path_to_objects + 'X_train_q2_w2v_vect.p', 'rb'))
q1_vecs_test = pickle.load(open(path_to_objects + 'X_test_q1_w2v_vect.p', 'rb'))
q2_vecs_test = pickle.load(open(path_to_objects + 'X_test_q2_w2v_vect.p', 'rb'))

In [ ]:
X_train['question1'] = q1_vecs_train
X_train['question2'] = q2_vecs_train
X_test['question1'] = q1_vecs_test
X_test['question2'] = q2_vecs_test

In [0]:
def text_to_word_list(text):
    ''' 
    Pre process and convert texts to a list of words 
    input: str
    output: list of cleaned word
    '''
    
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub("quikly","quickly", text)

    text = text.split()

    return text

In [0]:
questions_cols = ['question1', 'question2']
filters = [strip_tags, strip_multiple_whitespaces, remove_stopwords, stem_text]
# Iterate over the questions only of both training and test datasets
for dataset in [X_train, X_test]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in preprocess_string(row[question], filters):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions with lists of word indices
            dataset.set_value(index, question, q2n)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [0]:
max_seq_length = max(X_train.question1.map(lambda x:len(x)).max(),
                    X_train.question2.map(lambda x:len(x)).max(),
                    X_test.question1.map(lambda x:len(x)).max(),
                    X_test.question2.map(lambda x:len(x)).max())

In [0]:
len(vocabulary)

182150

In [0]:
embedding_dim = 300

embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  
embeddings[0] = 0 


# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

In [0]:
del word2vec

In [0]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

from keras.layers import Input, Bidirectional, LSTM, dot, Flatten, Dense, Reshape, add, Dropout, BatchNormalization
  
n_hidden = 50
gradient_clipping_norm  = 1.25

batch_size = 64
n_epoch = 6
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = layers.Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

shared_lstm = keras.layers.LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

distance = keras.layers.Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])


model = Model([left_input, right_input], [distance])

W0713 08:30:39.199456 140058315331456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0713 08:30:39.244539 140058315331456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0713 08:30:39.257477 140058315331456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0713 08:30:39.275491 140058315331456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0713 08:30:39.276655 1400583153

In [0]:
from keras import optimizers
optimizer = optimizers.Adam(clipnorm=gradient_clipping_norm)

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

W0713 08:30:40.646041 140058315331456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
from sklearn.model_selection import train_test_split
import itertools
# Split to train validation
validation_size = 40000
training_size = len(X_train) - validation_size

X = X_train[questions_cols]
Y = y_train

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size)

X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}

Y_train = Y_train.values
Y_validation = Y_validation.values

X_train['left'] = pad_sequences(X_train['left'],  maxlen=max_seq_length)
X_train['right'] = pad_sequences(X_train['right'],  maxlen=max_seq_length)
X_validation['left'] = pad_sequences(X_validation['left'],  maxlen=max_seq_length)
X_validation['right'] = pad_sequences(X_validation['right'],  maxlen=max_seq_length)
X_test['left'] = pad_sequences(X_test['left'],  maxlen=max_seq_length)
X_test['right'] = pad_sequences(X_test['right'],  maxlen=max_seq_length)

In [0]:
len(X_train['left'])

230872

In [0]:
from keras.callbacks import ModelCheckpoint
filepath = path_to_objects + "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, epochs=n_epoch, callbacks=callbacks_list,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

W0713 08:30:45.242915 140058315331456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 230872 samples, validate on 40000 samples
Epoch 1/6
230872/230872 [==============================] - 1240s 5ms/step - loss: 0.2100 - acc: 0.7220 - val_loss: 0.1760 - val_acc: 0.7470

Epoch 00001: val_acc improved from -inf to 0.74700, saving model to /content/gdrive/My Drive/UCU-2019-final-project-storage/serialization_objects/weights-improvement-01-0.75.hdf5
Epoch 2/6
230872/230872 [==============================] - 1205s 5ms/step - loss: 0.1645 - acc: 0.7655 - val_loss: 0.1633 - val_acc: 0.7685

Epoch 00002: val_acc improved from 0.74700 to 0.76850, saving model to /content/gdrive/My Drive/UCU-2019-final-project-storage/serialization_objects/weights-improvement-02-0.77.hdf5
Epoch 3/6
230872/230872 [==============================] - 1206s 5ms/step - loss: 0.1525 - acc: 0.7854 - val_loss: 0.1585 - val_acc: 0.7783

Epoch 00003: val_acc improved from 0.76850 to 0.77833, saving model to /content/gdrive/My Drive/UCU-2019-final-project-storage/serialization_objects/weights-improvem

In [0]:
!pip install h5py
import h5py
model.save(path_to_objects+"model.h5")

In [0]:
model.save_weights(path_to_objects+"model.h5")